In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from astropy.time import Time, TimeDelta

from lsst_efd_client import EfdClient

In [ ]:
client = EfdClient('summit_efd')

In [ ]:
# Times to start looking at the mount data
# We were slewing a small random distance, then tracking for 32 seconds
# We'll query the EFD during that time
start = Time("2022-11-29 00:00:00Z", scale='utc')
end = Time("2022-11-29 09:00:00Z", scale='utc')

In [ ]:
startTrack = await client.select_time_series('lsst.sal.MTMount.command_startTracking', \
                                                ['*'],  start, end)
stopTrack = await client.select_time_series('lsst.sal.MTMount.command_stopTracking', \
                                                ['*'],  start, end)

In [ ]:
# Only three - I thought there would be more ??
len(startTrack)

In [ ]:
startTrack

In [ ]:
# Plotting the mount plots
%matplotlib inline
track_counter = 2 # This chooses one of the tracking times
start = Time(startTrack.index[track_counter]) + TimeDelta(5.0, format='sec') # Need to shift to be sure mount has stabilized
end  = Time(stopTrack.index[track_counter])

az = await client.select_time_series('lsst.sal.MTMount.azimuth', \
                                            ['actualPosition'],  start, end)
el = await client.select_time_series('lsst.sal.MTMount.elevation', \
                                            ['actualPosition'],  start, end)    
# Calculate the tracking errors
az_vals = np.array(az.values[:,0])
el_vals = np.array(el.values[:,0])
times_az = az.index
times_el = el.index
# The fits are much better if the time variable
# is centered in the interval
fit_times_az = [(times_az[i]-times_az[int(len(az.values) / 2)]).total_seconds() for i in range(len(times_az))]
fit_times_el = [(times_el[i]-times_el[int(len(el.values) / 2)]).total_seconds() for i in range(len(times_el))]

# Fit with a polynomial
az_fit = np.polyfit(fit_times_az, az_vals, 4)
el_fit = np.polyfit(fit_times_el, el_vals, 4)
az_model = np.polyval(az_fit, fit_times_az)
el_model = np.polyval(el_fit, fit_times_el)

# Errors in arcseconds
az_error = (az_vals - az_model) * 3600
el_error = (el_vals - el_model) * 3600

# Calculate RMS
az_rms = np.sqrt(np.mean(az_error * az_error))
el_rms = np.sqrt(np.mean(el_error * el_error))

# Calculate Image impact RMS
# We are less sensitive to Az errors near the zenith
image_az_rms = az_rms * np.cos(el_vals[0] * np.pi / 180.0)
image_el_rms = el_rms 

fig = plt.figure(figsize = (8,8))
plt.subplots_adjust(wspace=0.3, hspace=0.5)
plt.suptitle(f"MT Mount Jitter - 20221128", fontsize = 18)
plt.subplot(2,2,1)
ax1 = az['actualPosition'].plot(legend=True, color='red')
ax1.set_title("Azimuth")
ax1.set_ylabel("Degrees")
plt.subplot(2,2,2)
ax3 = el['actualPosition'].plot(legend=True, color='green')
ax3.set_title("Elevation")
ax3.set_ylabel("Degrees")

plt.subplot(2,2,3)
plt.plot(fit_times_az, az_error, color='red')
plt.title(f"Azimuth RMS error = {az_rms:.2f} arcseconds\n"
          f"  Image RMS error = {image_az_rms:.2f} arcseconds", fontsize=10)
plt.ylim(-10.0,10.0)
plt.xticks([])
plt.ylabel("ArcSeconds")
plt.subplot(2,2,4)
plt.plot(fit_times_el, el_error, color='green')
plt.title(f"Elevation RMS error = {el_rms:.2f} arcseconds\n"
          f"  Image RMS error = {image_el_rms:.2f} arcseconds", fontsize=10)
plt.ylim(-10.0,10.0)
plt.xticks([])
plt.ylabel("ArcSeconds")
plt.savefig(f"/home/craiglagegit/DATA/MT_Mount_Jitter_20221128_{track_counter}.pdf")


In [ ]:
fig = plt.figure(figsize = (6,4))
plt.plot(fit_times_az[0:100], az_error[0:100], color='red')
plt.xlabel("Time from center of track (seconds)")
plt.ylabel("Azimuth Error (arcseconds)")
plt.ylim(-2,2)
plt.savefig(f"/home/craiglagegit/DATA/MT_Mount_Jitter_Blowup_20221128_{track_counter}.pdf")

In [ ]:
len(az_error)